In [14]:
%pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import re
import PyPDF2
import nltk
import os
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import spacy
import spacy_stanza
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import csv

In [16]:
nltk.download('punkt')
nlp = spacy_stanza.load_pipeline("id") 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khalf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-06-11 08:38:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-11 08:38:54 INFO: Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

2024-06-11 08:38:54 WARNING: GPU requested, but is not available!
2024-06-11 08:38:54 INFO: Using device: cpu
2024-06-11 08:38:54 INFO: Loading: tokenize
2024-06-11 08:39:01 INFO: Loading: mwt
2024-06-11 08:39:01 INFO: Loading: pos
2024-06-11 08:39:02 INFO: Loading: lemma
2024-06-11 08:39:02 INFO: Loading: constituency
2024-06-11 08:39:05 INFO: Loading: depparse
2024-06-11 08:39:06 INFO: Done loading processors!


In [27]:
def extract_text_from_pdf(pdf_path, exclude_patterns=None, start_page=5, end_page=11):
    text = ""
    if exclude_patterns is None:
        exclude_patterns = []

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        
        # Ensure the start and end page numbers are within the valid range
        #start_page = max(0, 0)  # Pages are 0-indexed in PyPDF2
        start_page = 0
        end_page = num_pages  # Ensure end_page does not exceed total pages
        
        for page_num in range(start_page, end_page):
            page = reader.pages[page_num]
            page_text = page.extract_text()

            # Exclude specified patterns from the extracted text
            for pattern in exclude_patterns:
                page_text = page_text.replace(pattern, '')

            text += page_text
    
    return text


In [28]:
# Define File Path
pdf_path = "C:/Users/khalf/Code/NLP/draft_ruu_kuhp_final.pdf"

# Extract File
extract_text = extract_text_from_pdf(pdf_path)
print(extract_text)

1 
  
 
RANCANGAN  
UNDANG -UNDANG REPUBLIK INDONESIA  
NOMOR … TAHUN …  
TENTANG  
KITAB UNDANG -UNDANG HUKUM PIDANA  
 
DENGAN RAHMAT TUHAN YANG MAHA ESA  
 
PRESIDEN REPUBLIK INDONESIA,  
 
Menimbang:  a. bahwa untuk mewujudkan hukum pidana nasional 
Negara Kesatuan Republik Indonesia yang berdasarkan 
Pancasila dan Undang -Undang Dasar Negara Republik 
Indonesia Tahun 1945  serta  asas hukum umum yang 
diakui masyarakat beradab , perlu disusun hukum pidana 
nasional untuk mengganti Kitab Undang -Undang Hukum 
Pidana warisan pemerintah kolonial Hindia Belanda;  
 b. bahwa hukum pidana nasional tersebut harus 
disesuaikan dengan politik hukum, keadaan, dan 
perkembangan kehidupan bermasyarakat, berbangsa , 
dan bernegara yang bertujuan menghormati dan 
menjunjung tinggi hak asasi manusia, berdasarkan 
Ketuhanan Yang Maha Esa, kemanusiaan yang adil dan 
beradab, persatuan Indonesia, kerakyatan yang dipimpin 
oleh hikmat kebijaksanaan dalam permusyawaratan/ 
perwakilan, dan keadilan so

In [33]:
def classify_text(text):
    bab_pattern = r'BAB\s+[IVXLCDM]+\s*.*'
    bagian_pattern = r'Bagian\s+[A-Za-z]+\s*.*'
    paragraf_pattern = r'Paragraf\s+\d+\s*.*'
    pasal_pattern = r'Pasal\s+\d+\s*.*'
    ayat_pattern = r'\(\d+\)\s*.*'

    data = []
    current_bab = current_bagian = current_paragraf = current_pasal = None

    lines = text.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        if re.match(bab_pattern, line):
            current_bab = line
            current_bagian = current_paragraf = current_pasal = None
        
        elif re.match(bagian_pattern, line):
            current_bagian = line
            current_paragraf = current_pasal = None
        
        elif re.match(paragraf_pattern, line):
            current_paragraf = line
            current_pasal = None
        
        elif re.match(pasal_pattern, line):
            current_pasal = line
            i += 1
            next_line = lines[i].strip() if i < len(lines) else ""
            
            # Check if the next line is an Ayat
            if re.match(ayat_pattern, next_line):
                while i < len(lines) and re.match(ayat_pattern, lines[i].strip()):
                    ayat_line = lines[i].strip()
                    data.append({
                        'BAB': current_bab,
                        'Bagian': current_bagian,
                        'Paragraf': current_paragraf,
                        'Pasal': current_pasal,
                        'Ayat': ayat_line
                    })
                    i += 1
            else:
                # No Ayat, just text under Pasal
                pasal_text = current_pasal + " " + next_line
                while i < len(lines) and not re.match(pasal_pattern, lines[i].strip()):
                    i += 1
                    if i < len(lines):
                        pasal_text += " " + lines[i].strip()
                data.append({
                    'BAB': current_bab,
                    'Bagian': current_bagian,
                    'Paragraf': current_paragraf,
                    'Pasal': current_pasal,
                    'Ayat': pasal_text
                })
                continue

        i += 1

    return pd.DataFrame(data)

In [34]:
Classifed_Text = classify_text(extract_text)
Classifed_Text.to_csv('classified to pasal.csv', index = False)
print(Classifed_Text.head)


<bound method NDFrame.head of             BAB         Bagian    Paragraf      Pasal  \
0         BAB I  Bagian Kesatu        None    Pasal 1   
1         BAB I  Bagian Kesatu        None    Pasal 2   
2         BAB I  Bagian Kesatu        None    Pasal 3   
3         BAB I   Bagian Kedua  Paragraf 1    Pasal 4   
4         BAB I   Bagian Kedua  Paragraf 1    Pasal 5   
...         ...            ...         ...        ...   
1276  BAB XXX V  Bagian Keenam        None  Pasal 628   
1277  BAB XXX V  Bagian Keenam        None  Pasal 629   
1278  BAB XXX V  Bagian Keenam        None  Pasal 630   
1279  BAB XXX V  Bagian Keenam        None  Pasal 631   
1280  BAB XXX V  Bagian Keenam        None  Pasal 632   

                                                   Ayat  
0     (1) Tidak ada satu perbuatan pun yang dapat di...  
1     (1)  Ketentuan sebagaimana  dimaksud dalam Pas...  
2     (1) Dalam hal terdapat perubahan peraturan per...  
3     Pasal 4 Ketentuan pidana dalam Undang -Undang .